In [1]:
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
path_JuanSe = "/Users/juan/Documents/Juan's MacBook Pro/CUNEF/Quinto/Informacion no Estructurada/Practica Final/prueba_3_info_no_estruct.xlsx"

df = pd.read_excel(path_JuanSe)

In [3]:
df

,title,price,info_features,comment,details
0,Piso en venta en calle de Don Ramón de la Cruz...,398.000 €,109 m²\n3 hab.\nPlanta 4ª interior con ascensor,¡Fantástica inversión! Descuento de aproximada...,Características básicas\n109 m² construidos\n3...
1,Dúplex en venta en Recoletos\nBarrio de Salama...,745.000 €,115 m²\n2 hab.\nPlanta 1ª exterior con ascensor,Excelente propiedad en Recoletos\nEngel & Völk...,Características básicas\n115 m² construidos\n2...
2,Piso en venta en Castellana\nBarrio de Salaman...,850.000 €,161 m²\n3 hab.\nPlanta 1ª interior con ascensor,Propiedad en la mejor zona de Castellana\nEnge...,"Características básicas\n161 m² construidos, 1..."
3,"Piso en venta en General Pardiñas\nGoya, Madri...",850.000 €,112 m²\n2 hab.\nPlanta 5ª exterior con ascensor,"Precioso piso exterior, muy luminoso, en barri...","Características básicas\n112 m² construidos, 9..."
4,"Piso en venta en calle de Lagasca\nRecoletos, ...",890.000 €,134 m²\n2 hab.\nSemi-sótano exterior con ascensor,Grupo Marsapi vende en EXCLUSIVA este increíbl...,Características básicas\n134 m² construidos\n2...
...,...,...,...,...,...
115,"Piso en venta en Lagasca\nCastellana, Madrid V...",2.800.000 €,300 m²\n3 hab.\nPlanta 6ª exterior con ascenso...,"Piso a reformar con balcones, mucha luz - 3 ha...",Características básicas\n300 m² construidos\n3...
116,"Piso en venta en calle de Castelló\nRecoletos,...",2.900.000 €,445 m²\n5 hab.\nPlanta 4ª exterior con ascenso...,Tailor & Key presenta elegante vivienda en una...,Características básicas\n445 m² construidos\n5...
117,Piso en venta en Castellana\nBarrio de Salaman...,2.940.000 €,452 m²\n6 hab.\nPlanta 4ª exterior con ascenso...,Magnífica y amplia vivienda ubicada en una de ...,Características básicas\n452 m² construidos\n6...
118,Piso en venta en Castellana\nBarrio de Salaman...,2.965.000 €,438 m²\n5 hab.\nPlanta 3ª exterior con ascenso...,PROMORA MADRID vende piso para reformar en bar...,Características básicas\n438 m² construidos\n5...


In [4]:
def symbol_remover(col, symbollist, num = True):
    """
    Esta funcion sirve para formatear columnas numericas que incluyen simbolos
    
    Input: 
    col : columna a formatear
    symbollist : lista de simbolos a eliminar
    
    Output: columna convertida en numerica libre de simbolos
    """
    
    for s in symbollist:
        col = col.str.replace(s, '')
    if num == True: 
        col = col.astype(float)
    
    return col

In [5]:
df["info_features"] = df["info_features"].str.split('\n')
df["details"] = df["details"].str.split('\n')


In [6]:
df["price"] = symbol_remover(df["price"], [".","€"])

In [7]:
df["m2"] = df["info_features"].str[0]
df["m2"] = symbol_remover(df["m2"], [" m²"])
df["info_features"]=df["info_features"].str[1:]

In [8]:
blank = []
for i in range(len(df)):
        if df["info_features"][i][0].endswith("."):
            blank.append(df["info_features"][i][0])
            del df["info_features"][i][0]
        else: blank.append(np.nan)

df["hab"] = blank
df["hab"] = symbol_remover(df["hab"], [" hab."])

In [9]:
for i in range(len(df)):
    df["info_features"][i][0] = df["info_features"][i][0].replace("Semi-sótano ", "Planta -1 ")
    df["info_features"][i][0] = df["info_features"][i][0].replace("Bajo ", "Planta 0 ")
    df["info_features"][i][0] = df["info_features"][i][0].replace("Entreplanta ", "Planta 0 ")
    if not df["info_features"][i][0].startswith("Planta "):
        df["info_features"][i][0] = df["info_features"][i][0].replace("exterior ", "Planta NA exterior ")

In [10]:
df["planta"] = df["info_features"].str[0]
df["planta"] = symbol_remover(df["planta"], ["Planta ", " ascensor", "ª"], num=False)
df["info_features"]=df["info_features"].str[1:]

In [11]:
df["exterior"] = df["planta"].str.contains('exterior')
df["ascensor"] = df["planta"].str.contains('con')
df["planta"] = df["planta"].str[:2]

In [12]:
blank = []
for i in range(len(df)):
    
    if len(df["info_features"][i]) == 0:
        blank.append(False)
            
    else: 
        if df["info_features"][i][0].startswith("G"):
                blank.append(True)
                del df["info_features"][i][0]
        else: blank.append(False)

df["garaje"] = blank


In [13]:
blank = []
for i in range(len(df)):
    if not len(df["info_features"][i]) == 0:
            if df["info_features"][i][0].startswith("Lu"):
                blank.append(True)
                del df["info_features"][i][0]
            else: blank.append(False)
    else: blank.append(False)

df["lujo"] = blank


In [14]:
blank = []
for i in range(len(df)):
    if not len(df["info_features"][i]) == 0:
            if df["info_features"][i][0].startswith("Lo"):
                blank.append(True)
                del df["info_features"][i][0]
            else: blank.append(False)
    else: blank.append(False)

df["loft"] = blank


In [15]:
del df["info_features"]

In [18]:
df.head()

,title,price,comment,details,m2,hab,planta,exterior,ascensor,garaje,lujo,loft
0,Piso en venta en calle de Don Ramón de la Cruz...,398000.0,¡Fantástica inversión! Descuento de aproximada...,"[Características básicas, 109 m² construidos, ...",109.0,3.0,4,False,True,False,False,False
1,Dúplex en venta en Recoletos\nBarrio de Salama...,745000.0,Excelente propiedad en Recoletos\nEngel & Völk...,"[Características básicas, 115 m² construidos, ...",115.0,2.0,1,True,True,False,False,False
2,Piso en venta en Castellana\nBarrio de Salaman...,850000.0,Propiedad en la mejor zona de Castellana\nEnge...,"[Características básicas, 161 m² construidos, ...",161.0,3.0,1,False,True,False,False,False
3,"Piso en venta en General Pardiñas\nGoya, Madri...",850000.0,"Precioso piso exterior, muy luminoso, en barri...","[Características básicas, 112 m² construidos, ...",112.0,2.0,5,True,True,False,False,False
4,"Piso en venta en calle de Lagasca\nRecoletos, ...",890000.0,Grupo Marsapi vende en EXCLUSIVA este increíbl...,"[Características básicas, 134 m² construidos, ...",134.0,2.0,-1,True,True,False,False,False
